<a href="https://colab.research.google.com/github/jonkstro/Challenges-LetsCode/blob/main/Agosto_Challenge_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Challenge: Sistema de recomendação

Suponha que você seja um cientista de dados que trabalha na área de vendas de uma empresa de locação de filmes.

Desse modo, você precisa recomendar filmes ainda não assistidos aos usuários que você possui em sua base de dados, de acordo com a similaridade entre os demais usuários.

Importante ressaltar que na primeira parte do trabalho, você e o seu time de ciência da dados devem realizar algumas etapas importantes, tais como:
* Tratamento dos dados;
* Entender o problema;
* Fazer suposições sobre os filmes e usuários a serem utilizados nos cálculos;
* Encontrar os usuários similares;
* Recomendar os itens ainda não vistos para um determinado usuário.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/Lets Code/Challenge Agosto

/content/drive/MyDrive/Colab Notebooks/Lets Code/Challenge Agosto


### IMPORTAR BIBLIOTECAS

In [ ]:
## BASEADO EM: https://www.kaggle.com/code/abjr002/movie-recommendation-system

# importando as bibliotecas e csv

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
# importar tabela de creditos do kaggle no drive
df_credits = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lets Code/Challenge Agosto/tmdb_5000_credits.csv')

df_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [ ]:
# importar tabela de filmes do kaggle no drive
df_movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lets Code/Challenge Agosto/tmdb_5000_movies.csv')

df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [ ]:
# realizar o merge das duas tabelas
filmes = df_movies.merge(df_credits, on='title')

filmes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

## ANALISE DOS DADOS:


### Removendo colunas irrelevantes e mantendo somente as necessárias pro mmodelo de negócios:

In [ ]:
filmes = filmes[['movie_id', 'title', 'genres', 'keywords', 'overview', 'cast', 'crew']]

filmes.head()

,movie_id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


### Removendo valores nulos [NA]

In [ ]:
# filmes.isnull().sum()
for col in filmes.columns:
  print(f"A coluna {col} possui {filmes[col].isnull().sum()} valores nulos")
  print("-" * 50)

A coluna movie_id possui 0 valores nulos
--------------------------------------------------
A coluna title possui 0 valores nulos
--------------------------------------------------
A coluna genres possui 0 valores nulos
--------------------------------------------------
A coluna keywords possui 0 valores nulos
--------------------------------------------------
A coluna overview possui 3 valores nulos
--------------------------------------------------
A coluna cast possui 0 valores nulos
--------------------------------------------------
A coluna crew possui 0 valores nulos
--------------------------------------------------


In [ ]:
# remover arquivos nulos 
filmes.dropna(inplace = True)

In [ ]:
# filmes.isnull().sum()
for col in filmes.columns:
  print(f"A coluna {col} possui {filmes[col].isnull().sum()} valores nulos")
  print("-" * 50)

A coluna movie_id possui 0 valores nulos
--------------------------------------------------
A coluna title possui 0 valores nulos
--------------------------------------------------
A coluna genres possui 0 valores nulos
--------------------------------------------------
A coluna keywords possui 0 valores nulos
--------------------------------------------------
A coluna overview possui 0 valores nulos
--------------------------------------------------
A coluna cast possui 0 valores nulos
--------------------------------------------------
A coluna crew possui 0 valores nulos
--------------------------------------------------


### Preenchendo valores duplicados

É possível observar que não temos valores duplicados no dataframe

In [ ]:
filmes.duplicated().sum()

0

## CRIAR NOVO DATAFRAME SIMPLIFICADO

### Extrair os generos dos filmes

In [ ]:
filmes.iloc[0]

movie_id                                                19995
title                                                  Avatar
genres      [{"id": 28, "name": "Action"}, {"id": 12, "nam...
keywords    [{"id": 1463, "name": "culture clash"}, {"id":...
overview    In the 22nd century, a paraplegic Marine is di...
cast        [{"cast_id": 242, "character": "Jake Sully", "...
crew        [{"credit_id": "52fe48009251416c750aca23", "de...
Name: 0, dtype: object

In [ ]:
filmes.iloc[0]['genres']

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [ ]:
# criar uma função de suporte na extração dos 'name' em genero e keywords:

import ast
def extrair(obj):
  l = []
  for i in ast.literal_eval(obj):
    l.append(i['name'])
  return l

In [ ]:
# aplicar a função para extração dos generos:
filmes['genres'] = filmes['genres'].apply(extrair)

In [ ]:
filmes['genres'].head(3)

0    [Action, Adventure, Fantasy, Science Fiction]
1                     [Adventure, Fantasy, Action]
2                       [Action, Adventure, Crime]
Name: genres, dtype: object

In [ ]:
filmes.iloc[0]['keywords']

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [ ]:
# iremos usar a mesma função de extrair para pegar a tag name do dict de keywords
filmes['keywords'] = filmes['keywords'].apply(extrair)

In [ ]:
filmes['keywords'].head(3)

0    [culture clash, future, space war, space colon...
1    [ocean, drug abuse, exotic island, east india ...
2    [spy, based on novel, secret agent, sequel, mi...
Name: keywords, dtype: object

In [ ]:
# da coluna crew, iremos pegar agora o diretor do filme
filmes.iloc[0]['crew']

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [ ]:
# criar uma função de suporte na extração dos diretores de 'crew':
## caso o job seja igual diretor, iremos pegar o nome do mesmo.
def extrair_diretor(obj):
  l = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      l.append(i["name"])
      break
  return l

In [ ]:
filmes['crew'] = filmes['crew'].apply(extrair_diretor)

In [ ]:
filmes['crew'].head(3)

0     [James Cameron]
1    [Gore Verbinski]
2        [Sam Mendes]
Name: crew, dtype: object

In [ ]:
# criar uma função de suporte na extração dos top 3 do elenco:
## caso o c seja menor que 3 iremos pegar o nome dos mesmos.
def extrair_elenco(obj):
  l = []
  c = 0
  for i in ast.literal_eval(obj):
    if c < 3:
      l.append(i["name"])
      c += 1
    else:
      break
  return l

In [ ]:
filmes['cast'] = filmes['cast'].apply(extrair_elenco)

In [ ]:
filmes['cast'].head(3)

0    [Sam Worthington, Zoe Saldana, Sigourney Weaver]
1       [Johnny Depp, Orlando Bloom, Keira Knightley]
2        [Daniel Craig, Christoph Waltz, Léa Seydoux]
Name: cast, dtype: object

### Agora será preciso transformar a overview em uma lista de strings para criarmos *tags* para manipulação

In [ ]:
# forma que se apresenta o overview:
filmes.iloc[0]['overview']

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [ ]:
# com a função abaixo iremos separar os overviews por espaço em branco,
## ex: jonas castro -> ['jonas', 'castro']
filmes['overview'] = filmes['overview'].apply(lambda x:x.split())

In [ ]:
filmes['overview'][0]

['In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.']

### TRANSFORMANDO AS VARIAVEIS
Iremos remover os espaços entre os nomes e sobrenomes e entre as palavras duplas nas keywords, pois pode causar alguma confusão na hora de treinar o modelo de machine learning.
Ex: Sam Worthington e Sam Mendes, ambos tem Sam no seu nome, o que pode causar problema no treino do modelo.

In [ ]:
# função para remover espaço entre os nomes
def remove_espaco(x):
  l = []
  for i in x:
    l.append(i.replace(' ', ''))
  return l

In [ ]:
filmes['cast'] = filmes['cast'].apply(remove_espaco)
filmes['crew'] = filmes['crew'].apply(remove_espaco)
filmes['genres'] = filmes['genres'].apply(remove_espaco)
filmes['keywords'] = filmes['keywords'].apply(remove_espaco)

In [ ]:
# novo df com os espacos removidos
filmes.head()

,movie_id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [ ]:
# criaremos agora a coluna tag no df
filmes['tags'] = filmes['overview'] + filmes['genres'] + filmes['keywords'] + filmes['cast'] + filmes['crew']
filmes.head(1)

,movie_id,title,genres,keywords,overview,cast,crew,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [ ]:
# criar novo df
new_data = filmes[['movie_id','title','tags']]
new_data.head(1)

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [ ]:
# separar novamente os textos para as tags do novo df

new_data['tags'] = new_data['tags'].apply(lambda x: ' '.join(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
new_data.head(1)

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."


### Stemming

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_data['tags'] = new_data['tags'].apply(stem)
new_data.head(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."


In [ ]:
new_data['tags'] = new_data['tags'].apply(lambda x: x.lower())
new_data.head(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."


In [ ]:
new_data['tags'][0]


'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

### Vetorização dos textos:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_data['tags']).toarray()

In [ ]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
vectors.shape

(4806, 5000)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(vectors)

In [ ]:
similarity_matrix

array([[1.        , 0.08458258, 0.08718573, ..., 0.04559608, 0.        ,
        0.        ],
       [0.08458258, 1.        , 0.06063391, ..., 0.02378257, 0.        ,
        0.02615329],
       [0.08718573, 0.06063391, 1.        , ..., 0.02451452, 0.        ,
        0.        ],
       ...,
       [0.04559608, 0.02378257, 0.02451452, ..., 1.        , 0.03962144,
        0.04229549],
       [0.        , 0.        , 0.        , ..., 0.03962144, 1.        ,
        0.08714204],
       [0.        , 0.02615329, 0.        , ..., 0.04229549, 0.08714204,
        1.        ]])

In [ ]:
sorted(list(enumerate(similarity_matrix[0])), reverse=True, key = lambda x:x[1])[1:6]

[(1216, 0.29061909685954823),
 (3730, 0.26401000024165),
 (507, 0.25903973506580724),
 (539, 0.2537477434955704),
 (2409, 0.2507061052819501)]

In [ ]:
def recomendar(movie):
    movie_index = new_data[new_data['title'] == movie].index[0]
    distances  = similarity_matrix[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key = lambda x:x[1])[1:6]
    
    for i in movie_list:
        print(new_data.iloc[i[0]].title)

## TESTANDO:

In [ ]:
recomendar('Avatar')

Aliens vs Predator: Requiem
Falcon Rising
Independence Day
Titan A.E.
Aliens


In [ ]:
new_data.head(5)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."
